<a href="https://colab.research.google.com/github/LOrdi15/Actividad-M1-Multiagentes/blob/main/ActividadM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Jose Andres Ordieres
#A01382904
#Aspiradoras


!pip install mesa matplotlib numpy pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector


%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams["animation.embed_limit"] = 2**128

import numpy as np
import pandas as pd
import random

import time
import datetime


## Imports
Agregar mesa, la plataforma para trabajar agentes.
- Importar mesa con Agent, Model
- Importar de mesa.space, Multigrid(varios agentes, en una sola celda).
- Importar de mesa.time, SimultaneousAxtivation, por que queremos que todos los agentes se muevan/hagan lo que tengan que hacer de manera simultanea o al mismo tiempo 
- Importar de mesa.datacollection, DataCollector para 

## Crear el Modelo


In [ ]:
def obtener_habitacion(modelo):
  habitacion = np.zeros((modelo.grid.width, modelo.grid.height))
  for celda in modelo.grid.coord_iter():
    contenido_celda, x, y = celda
    for contenido in contenido_celda:
      if isinstance(contenido, Aspiradora):
        habitacion[x][y] = 2
      else:
        habitacion[x][y] = contenido.estado

  return habitacion

class Aspiradora(Agent):

  def __init__(self, id_unico, modelo):
      super().__init__(id_unico, modelo)
      self.nueva_posicion = None
      self.movimientos = 0

  def step(self):
    vecinos = self.model.grid.get_neighbors(
        self.pos,
        moore=True,
        include_center=True)
    
    for vecino in vecinos:
      if isinstance(vecino, Piso) and vecino.pos == self.pos:
          vecino.siguiente_estado = vecino.estado
          if vecino.siguiente_estado == vecino.SUCIO:
            vecino.siguiente_estado = vecino.LIMPIO
            self.nueva_posicion = self.pos

          else:
            vecindario = self.model.grid.get_neighborhood(
                self.pos,
                moore=True,
                include_center=False)
            nueva_posicion = self.random.choice(vecindario)
            self.nueva_posicion = nueva_posicion
          break

          


  def advance(self):
    vecinos = self.model.grid.get_neighbors(
        self.pos,
        moore = False,
        include_center = True)
    
    for vecino in vecinos:
      if isinstance(vecino, Piso) and vecino.pos == self.pos:
        vecino.estado = vecino.siguiente_estado
        break
      
    if self.pos != self.nueva_posicion:
      self.movimientos = self.movimientos + 1

      #movemos la aspiradora a su nueva posicion
      self.model.grid.move_agent(self, self.nueva_posicion)

    
  

class Piso(Agent):

  SUCIO = 1
  LIMPIO = 0

  def __init__(self,pos,modelo,estado=LIMPIO):
    super().__init__(pos, modelo)
    self.x, self.y = pos
    self.estado = estado
    self.siguiente_estado = None
  

class Habitacion(Model):
  
  def __init__(self, m, n, num_agentes, por_celdas_sucias):
    self.num_agentes = num_agentes
    self.por_celdas_sucias = por_celdas_sucias
    self.por_celdas_limpias = 1 - por_celdas_sucias
    self.grid = MultiGrid(m, n, True)
    self.schedule = SimultaneousActivation(self)

    #Posicionar celdas sucias de forma aleatoria
    celdas_sucias = int ((m * n) * por_celdas_sucias)
    lista_celdas_vacias = list(self.grid.empties)
    for celdas in range(celdas_sucias):
      celda_vacia = random.choice(lista_celdas_vacias)
      piso = Piso(celda_vacia, self)
      piso.estado = piso.SUCIO
      self.grid.place_agent(piso, celda_vacia)
      self.schedule.add(piso)
      lista_celdas_vacias.remove(celda_vacia)

      #Posicionar celdas limpias
    lista_celdas_vacias = list(self.grid.empties)
    for celdas in lista_celdas_vacias:
      piso = Piso(celdas, self)
      self.grid.place_agent(piso, celdas)
      self.schedule.add(piso)
    
    for i in range(num_agentes):
      aspiradora = Aspiradora(i, self)
      self.grid.place_agent(aspiradora, (1,1))
      self.schedule.add(aspiradora)

    self.colectordatos = DataCollector(
        model_reporters = {"Habitacion": obtener_habitacion},
        agent_reporters = {"Movimientos": lambda a: getattr(a, "movimientos", None)}
    )

  def step(self):
    self.colectordatos.collect(self)
    self.schedule.step()

  def todasceldaslimpias(self):
    celdas_limpias = 0
    for celda in self.grid.coord_iter():
      contenido_celda, x, y = celda
      for contenido in contenido_celda:
        if isinstance(contenido, Piso) and contenido.estado == contenido.LIMPIO:      
          celdas_limpias = celdas_limpias + 1

    self.por_celdas_limpias = celdas_limpias / (self.grid.width * self.grid.height)
    if self.por_celdas_limpias == 1:
      return True
    
    else:
      return False


  

    

      


      


## Correr el Modelo


In [ ]:
# Datos de la habitacion:

M = 15
N = 15

# Numero de agentes
NUM_AGENTES = 10

#porcentaje de celdas inicialmente sucias:
PORCENTAJE_CELDAS_SUCIAS = 0.5

TIEMPO_MAXIMO_EJECUCION = 0.06

start_time = time.time()
tiempo_inicio = str(datetime.timedelta(seconds = TIEMPO_MAXIMO_EJECUCION))
modelo = Habitacion(M, N, NUM_AGENTES, PORCENTAJE_CELDAS_SUCIAS)

while((time.time() - start_time) < TIEMPO_MAXIMO_EJECUCION and not modelo.todasceldaslimpias()):
  modelo.step()

tiempo_ejecucion = str(datetime.timedelta(seconds=(time.time() - start_time)))

## Visualizacion


In [ ]:
todas_habitaciones = modelo.colectordatos.get_model_vars_dataframe()

In [ ]:
%%capture

fig, axs = plt.subplots(figsize = (5,5))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(todas_habitaciones.iloc[0][0], cmap="Greys")

def animate(i):
  patch.set_data(todas_habitaciones.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames=len(todas_habitaciones))

In [ ]:
anim

## Informe

In [ ]:
movimientos = modelo.colectordatos.get_agent_vars_dataframe()

print("Tiempo necesario hasta que todas las celdas esten limpias:", tiempo_ejecucion, "/", tiempo_inicio)
print("Porcentaje de celdas limpias despues del termino de la simulacion:", modelo.por_celdas_limpias)
print("Numero de movimiwntos realizados por todos los agentes:", movimientos.tail()["Movimientos"].sum())

Tiempo necesario hasta que todas las celdas esten limpias: 0:00:00.060608 / 0:00:00.060000
Porcentaje de celdas limpias despues del termino de la simulacion: 0.5644444444444444
Numero de movimiwntos realizados por todos los agentes: 26.0
